# WATER LOSS 

## Familiarization with the data

Imports and the libraries I will be using and data reading:

`“timeStamp”`: is the date time of the mesure.

`“Idflowmeter”`: is the ID of the meter The peculiar series of water measures for two sensor IDs are yet to be explained by Braila.

`“Tot1”`: is the water flow in m3

`“Tot2”`: is the water flow in m3. The difference between Tot1 and Tot2 are yet to be explained by Braila.

`“Analog2”`: is the water pressure of the measure point. The unit is probably Pascal, however not yet confirmed by Braila.

In [1]:
import math
import pandas as pandas
import numpy as np
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt

data = pandas.read_json("../data/water_loss_data_set_1.json")

Preparing data:
1. **Removed values that make no sense -> "0000-00-00 00:00:00", "2000-01-01 00:00:00"**
2. **Removed rows without tot1 data or without analog2 data**
3. **Formated column timeStamp to dateTime format**
4. **Floored minutes to the nearest hour**
5. **Divided data depending on the id of the measuring station**
6. **Prepared the data for plotting**

In [ ]:
# removing false values
data = data[(data.timeStamp != "0000-00-00 00:00:00" ) & (data.timeStamp != "2000-01-01 00:00:00")]

"""
Missing data in columns: 
tot1: 5086 
tot2: 100635 
analog2: 25

Removing data which is zero, be careful for 'analog2' only 25 values have zero. 
This could be values when some work was done on the pipes so the unhooked them -> pressure 0 ?
""" 
data = data[(data.tot1 != 0 ) & (data.analog2 != 0)]
data["timeStamp"] = pandas.to_datetime(data["timeStamp"], format="%Y-%m-%d %H:%M")

# COMMENT THIS IF YOU WANT MORE SPEED
data_with_minutes = data.copy(deep=True)
data_with_minutes["timeStamp"] = data_with_minutes["timeStamp"].dt.strftime("%Y-%m-%d %H:%M")

# flooring to the nearest hour and grouping by hour, and removing seconds and minutes
data["timeStamp"] = data["timeStamp"].dt.floor('h')
data["timeStamp"] = data["timeStamp"].dt.strftime("%Y-%m-%d %H:%M")
data["timeStamp"] = pandas.to_datetime(data["timeStamp"], format="%Y-%m-%d %H:%M")
data.sort_values("timeStamp")

# splitting the data by measuring station
data_249 = data[data['idflowmeter'] == "MAG8000_024905H318"]
data_248 = data[data['idflowmeter'] == "MAG8000_024805H318"]
data_with_minutes

Processing...

Removed this timestamps from data  
`['0000-00-00 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00']`

In [ ]:
data_249_by_hour = data_249.groupby("timeStamp").mean()
data_248_by_hour = data_248.groupby("timeStamp").mean()

data_249_by_hour.index = data_249_by_hour.index.strftime("%Y-%m-%d %H:%M")
data_248_by_hour.index = data_248_by_hour.index.strftime("%Y-%m-%d %H:%M")

data_249_by_hour

In [ ]:
data_248_by_hour

In [ ]:
timestamp_data_249 = data_249_by_hour.index.tolist()
timestamp_data_249 = [str(i) for i in timestamp_data_249]

timestamp_data_248 = data_248_by_hour.index.tolist()
timestamp_data_248 = [str(i) for i in timestamp_data_248]

y_axis_data = data.groupby("timeStamp").mean()[["analog2"]].to_numpy()
y_axis_data_249 = data_249_by_hour[["analog2"]].to_numpy()
y_axis_data_248 = data_248_by_hour[["analog2"]].to_numpy()

time :  
`['0000-00-00 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00', '2018-11-23 07:03:00', '2018-11-23 07:04:00', '2018-11-23 07:06:00', '2018-11-23 07:09:00', '2018-11-23 07:10:00']`
tot1 :  
`[-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]`

Max date: `2020-04-01 14:09:00`  
Min date: `0000-00-00 00:00:00` 
Median date: `2019-08-02 02:27:00` 

Max date: `2020-04-01 14:09:00` 
Min date: `0000-00-00 00:00:00` 
Median date: `2019-08-02 02:27:00`


False values:  `[['0000-00-00 00:00:00' -1.0]
 ['2000-01-01 00:00:00' 0.0]
 ['2000-01-01 00:00:00' 0.0]
 ['2000-01-01 00:00:00' 0.0]
 ['2000-01-01 00:00:00' 0.0]
 ['2018-11-23 07:03:00' 0.0]
 ['2018-11-23 07:04:00' 0.0]
 ['2018-11-23 07:06:00' 0.0]
 ['2018-11-23 07:09:00' 0.0]
 ['2018-11-23 07:10:00' 0.0]]`

Checking which data is missing.   
"tot2" attribute contains negative values mostly negative values!! Other do not...

In [ ]:
# data_248_by_hour.to_csv("id_248.csv", sep=';', encoding='utf-8')
# data_249_by_hour.to_csv("id_249.csv", sep=';', encoding='utf-8')

"""
DATA_SIZE = 1390649 
len_tot1 = len(data[(data['tot1'] != 0)])
len_tot2 = len(data[(data['tot2'] != 0)])
len_analog = len(data[(data['analog2'] != 0)])

print("Missing data in columns: ")
print("tot1:", (DATA_SIZE - len_tot1), "\ntot2:", (DATA_SIZE - len_tot2), "\nanalog2:", (DATA_SIZE- len_analog))
""";

In [ ]:
fig = plt.figure(figsize=(18, 12), dpi= 100, facecolor='w')
plt.plot(timestamp_data_249, y_axis_data_249, color="red")
plt.plot(timestamp_data_248, y_axis_data_248, color="blue")

plt.ylabel('Pressure in bar ?')
plt.xlabel('Date')
plt.xticks([i for i in range(0,len(y_axis_data), 1000)], rotation=40)
plt.rc('xtick',labelsize=8)
plt.title('Time')
#plt.savefig('test.png')
plt.show()

In [ ]:
fig = plt.figure(figsize=(18, 12), dpi= 100, facecolor='w')
plt.plot(timestamp_data_249, y_axis_data_249, color="red")

plt.ylabel('Pressure in bar ?')
plt.xlabel('Date')
plt.xticks([i for i in range(0,len(y_axis_data), 1000)], rotation=40)
plt.rc('xtick',labelsize=14)
plt.title('Pressure through time id - 249')
# plt.savefig('red_id.png')
plt.show()

In [ ]:
fig = plt.figure(figsize=(18, 12), dpi= 100, facecolor='w')
plt.plot(timestamp_data_248, y_axis_data_248, color="blue")

plt.ylabel('Pressure in bar ?')
plt.xlabel('Date')
plt.xticks([i for i in range(0,len(y_axis_data), 1000)], rotation=40)
plt.rc('xtick',labelsize=14)
plt.title('Pressure through time id - 248')
# plt.savefig('blue_id.png')
plt.show()

# Graph to check water pressure in a given timeframe

You have to choose two dates and the graph will be drawn between them. You must also choose the station for which you want to view the data.

In [ ]:
START_TIME = "2018-12-22 12:00"  
END_TIME = "2018-12-25 12:00"  
STATION = 248  

In [ ]:
def generate_timestamp_yaxis_array(start_time, end_time, station):
    choosen_data = data_248_by_hour
    
    if station == 249:
        choosen_data = data_249_by_hour
        
    choosen_data.index = pandas.to_datetime(choosen_data.index, format="%Y-%m-%d %H:%M")

    start_time = pandas.to_datetime(start_time, format="%Y-%m-%d %H:%M")
    end_time   = pandas.to_datetime(end_time, format="%Y-%m-%d %H:%M")
    choosen_data = choosen_data.loc[(choosen_data.index >= start_time) & (choosen_data.index <= end_time)]
    
    timestamp = choosen_data.index.tolist()
    timestamp = [str(i) for i in timestamp]
    
    y_axis = choosen_data[["analog2"]].to_numpy()
    
    fig = plt.figure(figsize=(18, 12), dpi= 100, facecolor='w')
    plt.plot(timestamp, y_axis, color="blue")

    plt.ylabel('Pressure in bar ?')
    plt.xlabel('Date')
    plt.xticks([i for i in range(0,len(y_axis), round(len(y_axis)/12))], rotation=40)
    plt.rc('xtick',labelsize=16)
    plt.title('Time')
    # plt.savefig('blue_id.png')
    plt.show()

In [ ]:
generate_timestamp_yaxis_array("2018-12-22 12:00", "2018-12-25 12:00", 248)

In [ ]:
generate_timestamp_yaxis_array("2019-05-11 12:00:00", "2019-06-13 12:00:00", 248)

### Daily view in minutes
Division again by id of the measuring device.

In [ ]:
data_minutes_249 = data_with_minutes[data_with_minutes['idflowmeter'] == "MAG8000_024905H318"]
data_minutes_248 = data_with_minutes[data_with_minutes['idflowmeter'] == "MAG8000_024805H318"]

data_minutes_249 = data_minutes_249.groupby("timeStamp").mean()
data_minutes_248 = data_minutes_248.groupby("timeStamp").mean()
data_minutes_248

In [ ]:
HOUR_ARRAY = [(f"{i:02d}" + ":00") for i in range(0,25)]

def generate_day_view(year_month_day, station):
    choosen_data = data_minutes_248
    
    if station == 249:
        choosen_data = data_minutes_249
    if len(choosen_data.index) < 1:
        raise Exception("No data for this date")
    
    start_day = year_month_day.strip() + " 00:00"
    end_day = year_month_day.strip() + " 23:59"
    choosen_data = choosen_data.loc[(choosen_data.index >= start_day) & (choosen_data.index <= end_day)]
    
    timestamp = choosen_data.index.tolist()
    timestamp = [str(i).replace(year_month_day, "").strip() for i in timestamp]
    
    y_axis = choosen_data[["analog2"]].to_numpy()
    
    fig = plt.figure(figsize=(18, 12), dpi= 100, facecolor='w')
    plt.plot(timestamp, y_axis, color="blue")
    plt.xticks(HOUR_ARRAY, rotation=40)
    plt.rc('xtick',labelsize=16)
    plt.margins(0, 0.1)
    
    plt.ylabel('Pressure in bar ?')
    plt.xlabel('Date')
    plt.title('Time')
    # plt.savefig('blue_id.png')
    plt.show()
    

In [ ]:
generate_day_view("2019-02-11", 248)

## Water flow in m3